In [17]:
import argparse
import os
import numpy as np
import json
from model.eval_voc import parse_voc_annotation
from model.eval_gen import BatchGenerator
from model.eval_utils import normalize, _sigmoid,makedirs,correct_yolo_boxes,do_nms,decode_netout,preprocess_input
from model.eval_utils import get_yolo_boxes,compute_overlap,compute_ap,_softmax
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import Adam
from keras.models import load_model
from model.yolo3 import yolo_body, tiny_yolo_body
from model.mobilenet import mobilenetv2_yolo_body
from keras.layers import Input

In [4]:
###############################
    #   Create the validation generator
    ###############################  
model_labels = [ "aeroplane", "bicycle", "bird","boat","bottle", "bus","car", "cat","chair", "cow", "diningtable", "dog","horse","motorbike","person", "pottedplant", "sheep","sofa", "train","tvmonitor"]
valid_ints, labels = parse_voc_annotation(
        "VOCdevkit/VOC2007/Annotations/", 
        "VOCdevkit/VOC2007/JPEGImages/", 
        "voc.pkl",
        model_labels
)

labels = labels.keys() if len(model_labels) == 0 else model_labels
labels = sorted(labels)

model_anchors = [ 10,13,  16,30,  33,23,  30,61,  62,45,  59,119,  116,90,  156,198,  373,326 ]
print("valid generator")
valid_generator = BatchGenerator(
        instances           = valid_ints, 
        anchors             = model_anchors,   
        labels              = labels,        
        downsample          = 32, # ratio between network input's size and network output's size, 32 for YOLOv3
        max_box_per_image   = 0,
        batch_size          = 2,
        min_net_size        = 288,
        max_net_size        = 448,   
        shuffle             = True, 
        jitter              = 0.0, 
        norm                = normalize
    )
os.environ['CUDA_VISIBLE_DEVICES'] = "0,1"

valid generator


In [5]:
infer_model = yolo_body(Input(shape=(None,None,3)), 3 , 20)

In [32]:
np.set_printoptions(threshold=np.inf)

In [27]:
#evaluate(infer_model, valid_generator)
model = infer_model 
generator = valid_generator 
iou_threshold=0.5
obj_thresh=0.5
nms_thresh=0.45
net_h=416
net_w=416
save_path=None

# gather all detections and annotations
all_detections     = [[None for i in range(generator.num_classes())] for j in range(10) ] # array of none
all_annotations    = [[None for i in range(generator.num_classes())] for j in range(10) ]


In [35]:
   # gather all detections and annotations
for i in range(10) :
        print(i)
        raw_image = [generator.load_image(i)]
        #print("generator = "+str(i) +" from " + str(generator.size()) )
        # make the boxes and the labels
        pred_boxes = get_yolo_boxes(model, raw_image, net_h, net_w, generator.get_anchors(), obj_thresh, nms_thresh)[0]
        
        #print("generator anchor = "+str(generator.get_anchors()) )
        score = np.array([box.get_score() for box in pred_boxes])
        print( score.shape )
        pred_labels = np.array([box.label for box in pred_boxes])        
        
        if len(pred_boxes) > 0:
            pred_boxes = np.array([[box.xmin, box.ymin, box.xmax, box.ymax, box.get_score()] for box in pred_boxes]) 
            print(pred_boxes)
        else:
            pred_boxes = np.array([[]])  
        
        # sort the boxes and the labels according to scores
        score_sort = np.argsort(-score)
        pred_labels = pred_labels[score_sort]
        pred_boxes  = pred_boxes[score_sort]
        
        # copy detections to all_detections
        for label in range(generator.num_classes()):
            #print("copy detections to all_detections  label "+str(label) )
            all_detections[i][label] = pred_boxes[pred_labels == label, :]

        annotations = generator.load_annotation(i)
        
        # copy detections to all_annotations
        for label in range(generator.num_classes()):
            #print("copy detections to all_annotations  label "+str(label) )
            all_annotations[i][label] = annotations[annotations[:, 4] == label, :4].copy()


0
(3037,)
[[ -49.  -99.   89.   12.    0.]
 [  -9.  -96.  127.    9.    0.]
 [  26.  -97.  168.   10.    0.]
 [-128. -248.  323.  160.    0.]
 [  67.  -96.  204.   10.    0.]
 [ -90. -250.  361.  162.    0.]
 [ 106.  -96.  242.   10.    0.]
 [ -53. -249.  400.  161.    0.]
 [ 144.  -96.  280.   10.    0.]
 [ -13. -248.  437.  159.    0.]
 [ 183.  -96.  319.   10.    0.]
 [  25. -249.  476.  161.    0.]
 [ 221.  -96.  357.   10.    0.]
 [  62. -247.  515.  159.    0.]
 [ 259.  -96.  396.   10.    0.]
 [ 101. -247.  553.  158.    0.]
 [ 297.  -96.  435.    9.    0.]
 [ 143. -246.  587.  158.    0.]
 [ 336.  -96.  473.    9.    0.]
 [ 184. -249.  624.  160.    0.]
 [ 372.  -93.  513.    8.    0.]
 [ 222. -250.  662.  162.    0.]
 [ 412.  -95.  549.    9.    0.]
 [ 265. -242.  697.  155.    0.]
 [ -51.  -61.   92.   51.    0.]
 [  -9.  -57.  129.   46.    0.]
 [  21.  -56.  175.   46.    0.]
 [  63.  -56.  210.   46.    0.]
 [ -87. -218.  358.  205.    0.]
 [ 102.  -57.  249.   47.    0.]


 [ 485.  414.  504.  450.    0.]]
1
(3026,)
[[ -48. -147.   89.  -33.    0.]
 [  -6. -142.  126.  -37.    0.]
 [  31. -144.  165.  -36.    0.]
 [-127. -300.  322.  117.    0.]
 [  72. -144.  201.  -36.    0.]
 [ 109. -145.  241.  -35.    0.]
 [ 149. -145.  278.  -35.    0.]
 [ 187. -145.  317.  -35.    0.]
 [ 225. -145.  355.  -35.    0.]
 [ 263. -145.  394.  -35.    0.]
 [ 301. -144.  433.  -36.    0.]
 [ 340. -144.  471.  -36.    0.]
 [ 375. -139.  512.  -39.    0.]
 [ 413. -143.  549.  -36.    0.]
 [ -51. -111.   93.    6.    0.]
 [  -6. -106.  128.    0.    0.]
 [  27. -106.  172.    1.    0.]
 [  66. -106.  210.    1.    0.]
 [ 105. -106.  249.    2.    0.]
 [ 144. -106.  286.    2.    0.]
 [  -4. -284.  428.  175.    0.]
 [ 183. -106.  325.    2.    0.]
 [  34. -285.  465.  175.    0.]
 [ 221. -106.  363.    2.    0.]
 [  74. -284.  502.  174.    0.]
 [ 258. -105.  402.    2.    0.]
 [ 113. -284.  540.  175.    0.]
 [ 297. -105.  441.    1.    0.]
 [ 155. -283.  575.  174.    0.]

 [ 485.  367.  504.  403.    0.]]
2
(3051,)
[[ -70.  -24.   17.   48.    0.]
 [ -44.  -21.   40.   45.    0.]
 [ -19.  -22.   65.   46.    0.]
 [   5.  -21.   90.   46.    0.]
 [ -94. -130.  188.  152.    0.]
 [  29.  -21.  115.   46.    0.]
 [  54.  -20.  139.   45.    0.]
 [  78.  -22.  164.   46.    0.]
 [ 103.  -22.  187.   46.    0.]
 [ 128.  -22.  212.   47.    0.]
 [ 152.  -22.  237.   46.    0.]
 [ 176.  -22.  262.   46.    0.]
 [  76. -123.  362.  145.    0.]
 [ 199.  -19.  288.   44.    0.]
 [ 104. -125.  382.  148.    0.]
 [ 224.  -21.  312.   46.    0.]
 [ 129. -115.  406.  139.    0.]
 [ -71.    0.   19.   72.    0.]
 [ -45.    0.   43.   71.    0.]
 [ -24.    2.   72.   70.    0.]
 [   0.    3.   97.   70.    0.]
 [  24.    2.  122.   71.    0.]
 [ -59. -117.  200.  186.    0.]
 [  49.    3.  146.   71.    0.]
 [ -36. -120.  227.  190.    0.]
 [  74.    2.  170.   71.    0.]
 [ -17. -118.  257.  188.    0.]
 [ 101.    2.  192.   71.    0.]
 [   5. -113.  283.  185.    0.]

 [ 270.  305.  283.  328.    0.]]
3
(3001,)
[[-131.  -37.    7.   75.    0.]
 [ -89.  -33.   44.   71.    0.]
 [-242. -192.  194.  227.    0.]
 [ -52.  -34.   84.   72.    0.]
 [ -14.  -33.  122.   72.    0.]
 [-169. -199.  276.  234.    0.]
 [  24.  -34.  160.   72.    0.]
 [  64.  -33.  197.   72.    0.]
 [ 103.  -35.  236.   74.    0.]
 [ 142.  -38.  273.   75.    0.]
 [ 183.  -37.  310.   74.    0.]
 [ 219.  -35.  351.   72.    0.]
 [ 256.  -34.  391.   72.    0.]
 [ 290.  -28.  432.   68.    0.]
 [ 331.  -33.  467.   72.    0.]
 [ 181. -183.  615.  219.    0.]
 [-133.    0.   10.  113.    0.]
 [-171.  -63.   43.  176.    0.]
 [ -92.    1.   48.  111.    0.]
 [ -59.    4.   93.  110.    0.]
 [ -20.    4.  132.  110.    0.]
 [  19.    4.  168.  111.    0.]
 [-119. -173.  301.  284.    0.]
 [  57.    4.  207.  112.    0.]
 [ -75. -183.  334.  293.    0.]
 [  99.    4.  242.  111.    0.]
 [ -38. -179.  373.  290.    0.]
 [ 140.    2.  278.  111.    0.]
 [ -11. -170.  423.  282.    0.]

 [ 402.  477.  422.  513.    0.]]
4
(3032,)
[[ -48. -121.   89.   -8.    0.]
 [  -7. -117.  127.  -11.    0.]
 [  31. -119.  165.   -9.    0.]
 [  72. -119.  201.   -9.    0.]
 [ 110. -119.  241.   -9.    0.]
 [ 149. -120.  278.   -8.    0.]
 [ 187. -120.  317.   -8.    0.]
 [ 226. -120.  355.   -8.    0.]
 [ 265. -119.  393.   -9.    0.]
 [ 302. -118.  432.  -10.    0.]
 [ 341. -119.  471.  -10.    0.]
 [ 376. -113.  512.  -13.    0.]
 [ 414. -116.  549.  -10.    0.]
 [ -51.  -84.   94.   31.    0.]
 [  -7.  -81.  128.   27.    0.]
 [  28.  -80.  171.   27.    0.]
 [  67.  -80.  209.   28.    0.]
 [ 105.  -79.  249.   28.    0.]
 [ -43. -254.  390.  197.    0.]
 [ 143.  -80.  287.   29.    0.]
 [   0. -256.  422.  198.    0.]
 [ 182.  -81.  325.   29.    0.]
 [  37. -256.  462.  199.    0.]
 [ 222.  -80.  362.   29.    0.]
 [  75. -257.  501.  199.    0.]
 [ 260.  -80.  401.   28.    0.]
 [ 114. -258.  539.  201.    0.]
 [ 298.  -79.  440.   27.    0.]
 [ 158. -257.  571.  200.    0.]

...

 [ 485.  381.  504.  417.    0.]]
6
(3025,)
[[-111.  -37.   26.   75.    0.]
 [ -71.  -33.   64.   72.    0.]
 [ -33.  -35.  103.   73.    0.]
 [ -70.  -92.  137.  129.    0.]
 [-194. -187.  263.  224.    0.]
 [   6.  -36.  140.   74.    0.]
 [-151. -189.  297.  224.    0.]
 [  44.  -36.  180.   73.    0.]
 [-105. -188.  328.  224.    0.]
 [  82.  -36.  220.   74.    0.]
 [ -66. -191.  367.  227.    0.]
 [ 121.  -35.  258.   74.    0.]
 [ -33. -197.  411.  232.    0.]
 [ 160.  -35.  295.   73.    0.]
 [  -3. -197.  458.  232.    0.]
 [ 199.  -33.  333.   71.    0.]
 [  38. -197.  493.  232.    0.]
 [ 237.  -32.  372.   71.    0.]
 [  81. -195.  526.  229.    0.]
 [ 274.  -31.  411.   70.    0.]
 [ 309.  -28.  452.   68.    0.]
 [ 161. -196.  599.  231.    0.]
 [ 205. -186.  632.  223.    0.]
 [-114.    0.   30.  114.    0.]
 [ -71.    4.   66.  109.    0.]
 [ -37.    4.  110.  110.    0.]
 [   3.    4.  146.  110.    0.]
 [  43.    3.  183.  110.    0.]
 [  81.    4.  223.  110.    0.]

 [ 422.  477.  442.  513.    0.]]
7
(3016,)
[[-132.  -37.    5.   74.    0.]
 [ -93.  -32.   44.   71.    0.]
 [ -56.  -33.   84.   72.    0.]
 [-213. -186.  240.  223.    0.]
 [ -16.  -34.  121.   72.    0.]
 [  23.  -35.  157.   74.    0.]
 [-137. -185.  317.  222.    0.]
 [  62.  -35.  195.   74.    0.]
 [ 101.  -35.  234.   73.    0.]
 [ 140.  -36.  271.   74.    0.]
 [ 178.  -35.  311.   74.    0.]
 [ 215.  -34.  351.   72.    0.]
 [  61. -190.  504.  226.    0.]
 [ 254.  -35.  390.   73.    0.]
 [ 290.  -30.  429.   70.    0.]
 [ 141. -196.  576.  232.    0.]
 [ 328.  -33.  467.   72.    0.]
 [ 180. -180.  615.  217.    0.]
 [-135.    0.    9.  113.    0.]
 [ -93.    5.   45.  108.    0.]
 [ -60.    5.   89.  110.    0.]
 [-207. -163.  233.  276.    0.]
 [ -18.    5.  124.  110.    0.]
 [-169. -165.  271.  278.    0.]
 [  21.    3.  161.  112.    0.]
 [-136. -157.  315.  271.    0.]
 [  63.    2.  197.  113.    0.]
 [ -90. -148.  346.  261.    0.]
 [ 100.    2.  237.  111.    0.]

 [ 401.  477.  421.  513.    0.]]
8
(3020,)
[[-119.  -37.   17.   75.    0.]
 [ -78.  -34.   54.   71.    0.]
 [-231. -190.  206.  226.    0.]
 [ -41.  -34.   94.   73.    0.]
 [-196. -191.  249.  227.    0.]
 [  -3.  -34.  133.   73.    0.]
 [-160. -197.  290.  232.    0.]
 [  35.  -34.  171.   72.    0.]
 [-129. -198.  334.  233.    0.]
 [  75.  -34.  208.   73.    0.]
 [ -92. -197.  375.  231.    0.]
 [ 113.  -34.  247.   72.    0.]
 [ -52. -195.  412.  229.    0.]
 [ 152.  -34.  285.   72.    0.]
 [ -12. -197.  449.  230.    0.]
 [ 190.  -34.  324.   72.    0.]
 [  28. -197.  485.  231.    0.]
 [ 228.  -33.  362.   71.    0.]
 [  73. -198.  517.  232.    0.]
 [ 266.  -33.  402.   71.    0.]
 [ 301.  -28.  443.   68.    0.]
 [ 195. -187.  623.  223.    0.]
 [-122.    0.   21.  113.    0.]
 [-160.  -62.   53.  175.    0.]
 [ -80.    2.   58.  110.    0.]
 [ -48.    5.  104.  109.    0.]
 [  -8.    5.  141.  109.    0.]
 [  29.    6.  181.  109.    0.]
 [  68.    6.  219.  109.    0.]

 [ 413.  477.  433.  513.    0.]]
9
(3026,)
[[-131.  -37.    4.   75.    0.]
 [-162.  -89.   32.  125.    0.]
 [ -90.  -34.   41.   72.    0.]
 [-242. -191.  192.  227.    0.]
 [ -52.  -35.   81.   73.    0.]
 [-205. -193.  233.  228.    0.]
 [ -15.  -35.  121.   73.    0.]
 [-170. -202.  275.  236.    0.]
 [  23.  -34.  158.   73.    0.]
 [  63.  -34.  196.   72.    0.]
 [-110. -198.  368.  232.    0.]
 [ 103.  -36.  233.   73.    0.]
 [ -69. -193.  404.  227.    0.]
 [ 142.  -35.  271.   73.    0.]
 [ -24. -198.  436.  232.    0.]
 [ 179.  -34.  311.   73.    0.]
 [  20. -206.  468.  239.    0.]
 [ 217.  -33.  350.   71.    0.]
 [ 256.  -33.  389.   71.    0.]
 [ 288.  -28.  431.   68.    0.]
 [ 182. -189.  612.  225.    0.]
 [-133.    1.    7.  112.    0.]
 [-174.  -63.   43.  176.    0.]
 [ -93.    1.   47.  111.    0.]
 [ -62.    4.   93.  109.    0.]
 [ -21.    5.  130.  110.    0.]
 [  16.    7.  169.  109.    0.]
 [  57.    6.  206.  110.    0.]
 [ -78. -187.  335.  297.    0.]

 [ 401.  477.  420.  513.    0.]]


In [29]:
len(all_detections )

10

In [ ]:
# compute mAP by comparing all detections and all annotations
average_precisions = {}
for label in  range(10) :
        false_positives = np.zeros((0,))
        true_positives  = np.zeros((0,))
        scores          = np.zeros((0,))
        num_annotations = 0.0

        #print("get average precisions every label"+str(label) )

        for i in range(10):
            detections           = all_detections[i][label]
            annotations          = all_annotations[i][label]
            num_annotations     += annotations.shape[0]
            detected_annotations = []

            #print("group all detection "+str(generator) )

            for d in detections:
                #print("detection group"+str(d) )
                scores = np.append(scores, d[4])

                if annotations.shape[0] == 0:
                    false_positives = np.append(false_positives, 1)
                    true_positives  = np.append(true_positives, 0)
                    print( "true_positive = {}".format(true_positives) )
                    continue

                overlaps            = compute_overlap(np.expand_dims(d, axis=0), annotations)
                assigned_annotation = np.argmax(overlaps, axis=1)
                max_overlap         = overlaps[0, assigned_annotation]

                if max_overlap >= iou_threshold and assigned_annotation not in detected_annotations:
                    false_positives = np.append(false_positives, 0)
                    true_positives  = np.append(true_positives, 1)
                    detected_annotations.append(assigned_annotation)
                    
                    print( "true_positive = {}".format(true_positives) )
                else:
                    false_positives = np.append(false_positives, 1)
                    true_positives  = np.append(true_positives, 0)
                    
                    print( "true_positive = {}".format(true_positives) )
                    

        # no annotations -> AP for this class is 0 (is this correct?)
        if num_annotations == 0:
            average_precisions[label] = 0
            continue

        # sort by score
        indices         = np.argsort(-scores)
        false_positives = false_positives[indices]
        true_positives  = true_positives[indices]
        print( "true_positive = {}".format(true_positives) )

        # compute false positives and true positives
        false_positives = np.cumsum(false_positives)
        true_positives  = np.cumsum(true_positives)
        print( "true_positive = {}".format(true_positives) )

        # compute recall and precision
        recall    = true_positives / num_annotations
        precision = true_positives / np.maximum(true_positives + false_positives, np.finfo(np.float64).eps)
        print( "recall = {}".format(recall) )
        print( "precision = {}".format(precision) )
        # compute average precision
        average_precision  = compute_ap(recall, precision)  
        average_precisions[label] = average_precision
average_precisions